DESARROLLO API

In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.utils.extmath import randomized_svd
from sklearn.feature_extraction.text import TfidfVectorizer
import zipfile

In [3]:
 # Sacamos el Data frame zipiado
def extract_data_from_zip(zip_file):
   
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall('movies')
    # Lectura de data frame
    movies_df = pd.read_csv('movies/movies.csv')
    return movies_df

In [4]:
df = pd.read_csv('movies.csv',encoding='utf-8')

CANTIDAD FILMACIONES POR MES
    
def cantidad_filmaciones_mes( Mes ): Se ingresa un mes en idioma Español. Debe devolver la cantidad de películas que fueron estrenadas en el mes consultado en la totalidad del datase

In [5]:
# Función de Películas por Mes
def peliculas_mes(mes:str):
     
    mes = mes.lower()
    meses = {
    'enero': 1,
    'febrero': 2,
    'marzo': 3,
    'abril': 4,
    'mayo': 5,
    'junio': 6,
    'julio': 7,
    'agosto': 8,
    'septiembre': 9,
    'octubre': 10,
    'noviembre': 11,
    'diciembre': 12}

    mes_numero = meses[mes]

    # Convertir la columna "fecha" a un objeto de tipo fecha
    df['release_date'] = pd.to_datetime(df['release_date'])

    # Tratamos la excepciòn
    try:
        month_filtered = df[df['release_date'].dt.month == mes_numero]
    except (ValueError, KeyError, TypeError):
        return None

    # Filtramos valores duplicados del dataframe y calculamos
    month_unique = month_filtered.drop_duplicates(subset='id')
    respuesta = month_unique.shape[0]

    return {'mes':mes, 'cantidad de peliculas':respuesta}

In [6]:
peliculas_mes('septiembre')

{'mes': 'septiembre', 'cantidad de peliculas': 4834}

CANTIDAD DE FILMACIONES POR DIA

def cantidad_filmaciones_dia( Dia ): Se ingresa un día en idioma Español. Debe devolver la cantidad de películas que fueron estrenadas en día consultado en la totalidad del dataset.

In [7]:
# Función de Películas por Día
def peliculas_dia(dia:str):
    
    days = {'lunes': 'Monday', 'martes': 'Tuesday', 'miercoles': 'Wednesday', 'jueves': 'Thursday', 'viernes': 'Friday', 'sabado': 'Saturday', 'domingo': 'Sunday'}
    day = days[dia.lower()]
    lista_peliculas_day = df[df['release_date'].dt.day_name() == day].drop_duplicates(subset='id')
    respuesta = lista_peliculas_day.shape[0]

    return {'dia': dia, 'cantidad de peliculas': respuesta}

In [8]:
peliculas_dia('martes')

{'dia': 'martes', 'cantidad de peliculas': 4639}

SCORE POR TITULO DE LA FILMACION

def score_titulo( titulo_de_la_filmación ): Se ingresa el título de una filmación esperando como respuesta el título, el año de estreno y el score

In [9]:
#Funcion Score por Pelicula
def score_titulo(pelicula):
    
    pelicula = pelicula.lower()

    info_pelicula = df[df['title'].str.lower() == pelicula].drop_duplicates(subset='title')

    pelicula_nombre = info_pelicula['title'].iloc[0]
    year_pelicula = str(info_pelicula['release_year'].iloc[0])
    score_pelicula = info_pelicula['vote_average'].iloc[0]

    return {'pelicula': pelicula_nombre, 'año': year_pelicula, "score": score_pelicula}

In [24]:
score_titulo('Star wars')

{'pelicula': 'Star Wars', 'año': '1977', 'score': 8.1}

VOTOS POR TITULO DE LA FILMACION

def votos_titulo( titulo_de_la_filmación ): Se ingresa el título de una filmación esperando como respuesta el título, la cantidad de votos y el valor promedio de las votaciones. La misma variable deberá de contar con al menos 2000 valoraciones, caso contrario, debemos contar con un mensaje avisando que no cumple esta condición y que por ende, no se devuelve ningun valor.

In [11]:
#Funcion Votos por Titulo
def votos_titulo(pelicula):
        
    pelicula = pelicula.lower()

    info_pelicula = df[df['title'].str.lower() == pelicula].drop_duplicates(subset='title')

    pelicula_nombre = info_pelicula['title'].iloc[0]
    vote_pelicula = info_pelicula['vote_count'].iloc[0]
    score_pelicula = info_pelicula['vote_average'].iloc[0]
    if vote_pelicula < 2000:
        return 'La Pelicula debe tener mas de 2000 valoraciones'
    return {'pelicula': pelicula_nombre, 'votos': vote_pelicula, "score": score_pelicula}

In [23]:
votos_titulo("zapped")

'La Pelicula debe tener mas de 2000 valoraciones'

In [14]:
votos_titulo('Star Wars')

{'pelicula': 'Star Wars', 'votos': 6778.0, 'score': 8.1}

EXITO ACTOR

def get_actor( nombre_actor ): Se ingresa el nombre de un actor que se encuentre dentro de un dataset debiendo devolver el éxito del mismo medido a través del retorno. Además, la cantidad de películas que en las que ha participado y el promedio de retorno. La definición no deberá considerar directores.

In [15]:
# Funcion Exito Actor
def get_actor(actor):

    actor = actor.lower()
   
    lista_get_actor = df[df['nombre_actor'].str.lower() == actor].drop_duplicates(subset='id')
    cantidad = (lista_get_actor).shape[0]
    retorno = (lista_get_actor['return'].iloc[0])

    promedio_retorno = int(retorno)*10000000/int(cantidad)

    return {'actor':actor, 'cantidad peliculas':cantidad, 'promedio retorno': promedio_retorno}

In [16]:
get_actor('Tom Hanks')

{'actor': 'tom hanks',
 'cantidad peliculas': 42,
 'promedio retorno': 2857142.8571428573}

EXITO DIRECTOR

def get_director( nombre_director ): Se ingresa el nombre de un director que se encuentre dentro de un dataset debiendo devolver el éxito del mismo medido a través del retorno. Además, deberá devolver el nombre de cada película con la fecha de lanzamiento, retorno individual, costo y ganancia de la misma.

In [17]:
#Funcion Exito Director
def get_director(director):

    director = director.lower()

    lista_get_director = df[df['nombre_director'].str.lower() == director].drop_duplicates(subset='id')
    retorno = lista_get_director['return'].sum() / lista_get_director['budget'].sum()
    pelicula = lista_get_director[['title', 'release_year', 'return', 'budget', 'revenue']].to_dict('records')
    
    return {'director': director, 'retorno': retorno,'pelicula,': pelicula}

In [18]:
get_director('John Lasseter')

{'director': 'john lasseter',
 'retorno': 4.0979861757860196e-08,
 'pelicula,': [{'title': 'Toy Story',
   'release_year': 1995,
   'return': 12.4518011,
   'budget': 30000000,
   'revenue': 373554033.0},
  {'title': 'The Incredibles',
   'release_year': 2004,
   'return': 6.863501,
   'budget': 92000000,
   'revenue': 631442092.0},
  {'title': "Howl's Moving Castle",
   'release_year': 2004,
   'return': 9.779602291666668,
   'budget': 24000000,
   'revenue': 234710455.0},
  {'title': 'Luxo Jr.',
   'release_year': 1986,
   'return': 0.0,
   'budget': 0,
   'revenue': 0.0},
  {'title': 'Cars',
   'release_year': 2006,
   'return': 3.849859575,
   'budget': 120000000,
   'revenue': 461983149.0},
  {'title': 'Bolt',
   'release_year': 2008,
   'return': 2.0665332933333334,
   'budget': 150000000,
   'revenue': 309979994.0},
  {'title': 'The Princess and the Frog',
   'release_year': 2009,
   'return': 2.543293,
   'budget': 105000000,
   'revenue': 267045765.0},
  {'title': 'Tangled',
 

RECOMENDACION

Se ingresa el nombre de una película y te recomienda las similares en una lista de 5 valores.

In [19]:
#Funcion de Recomendacion de Peliculas
def calculate_reduced_similarity_matrix(df):
    # El Vectorizador TfidfVectorizer con parámetros de reduccion procesamiento
    df['genres'].fillna('', inplace=True)
    vectorizar = TfidfVectorizer(min_df=10, max_df=0.5, ngram_range=(1,2))

    # Vectorizamos, ajustamos y transformamos el texto de la columna "title" del DataFrame
    X = vectorizar.fit_transform(df['genres'])

    # Calcular la matriz de similitud de coseno con una matriz reducida de 7000
    similarity_matrix = cosine_similarity(X[:1250,:])

    # Obtenemos la descomposición en valores singulares aleatoria de la matriz de similitud de coseno con 10 componentes
    n_components = 10
    U, Sigma, VT = randomized_svd(similarity_matrix, n_components=n_components)

    # Construir la matriz reducida de similitud de coseno
    reduced_similarity_matrix = U.dot(np.diag(Sigma)).dot(VT)

    return reduced_similarity_matrix
# Inicializar y cargar el DataFrame df
df = pd.read_csv("./movies.csv",encoding='utf-8')

# Calcular la matriz de similitud reducida
reduced_similarity_matrix = calculate_reduced_similarity_matrix(df)

c:\Users\crica\AppData\Local\Programs\Python\Python37-32\lib\site-packages\sklearn\utils\extmath.py:376: FutureWarning: If 'random_state' is not supplied, the current default is to use 0 as a fixed seed. This will change to  None in version 1.2 leading to non-deterministic results that better reflect nature of the randomized_svd solver. If you want to silence this warning, set 'random_state' to an integer seed or to None explicitly depending if you want your code to be deterministic or not.
  FutureWarning,


In [20]:
#Funcion de Recomendacion de Peliculas
def recomendacion(titulo:str):
    '''Ingresas un nombre de pelicula y te recomienda las similares en una lista'''
    
    titulo = titulo.title()
    #Ubicamos el indice del titulo pasado como parametro en la columna 'title' del dts user_item
    indice = np.where(df['title'] == titulo)[0][0]
    #Vemos los indices de aquellas puntuaciones y caracteristicas similares hacia el titulo 
    puntuaciones_similitud = reduced_similarity_matrix[indice,:]
    # Se ordena los indicies de menor a mayor
    puntuacion_ordenada = np.argsort(puntuaciones_similitud)[::-1]
    # Que solo 5 nos indique 
    top_indices = puntuacion_ordenada[:5]
    
    return df.loc[top_indices, 'title'].tolist()

In [21]:
recomendacion('Toy Story')

['Home for the Holidays',
 'Ace Ventura: When Nature Calls',
 'Babe',
 'Ace Ventura: When Nature Calls',
 'Ace Ventura: When Nature Calls']